Rescraping earliest_release_location_opening_gross and earliest_release_location_original_gross

In [ ]:
from bs4 import BeautifulSoup
import requests
import time, os
import re
import pandas as pd
import time
import pickle

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
earliestrelease_notdomestic_movies2 = pd.read_pickle('earliestrelease_notdomestic_movies2.pickle')

In [ ]:
def get_movie_dict(link):
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    for i in range(5): 
        try:
            response = requests.get(url)
        except:
            time.sleep(10) 
            continue
        else:
            break
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    headers = ['title', 
              'earliest_release_location_opening_gross',
               'earliest_release_location_original_gross']
    #Get title
    title_string = soup.find('title').text
    title = title_string.split('- Box Office Mojo')[0].strip()
    
    #OPENING CHROME DRIVER 
    driver = webdriver.Chrome(chromedriver)
    for i in range(5): #Try 5 times to reach page
        try:
            driver.get(url)
        except:
            time.sleep(10) # wait 10 seconds and then make http request again
            continue
        else:
            break
    
    #CLICKING ON RELEASE GROUP DROPDOWN AND SELECTING ORIGINAL
    for i in range(5): #Try 5 times to click dropdown
        try:
            all_releases_dropdown = driver.find_element_by_xpath('//*[@id="releasegroup-picker-navSelector"]')
            all_releases_dropdown.send_keys("Original Release")
        except:
            time.sleep(5) # wait 5 seconds and then make request again
            driver.refresh() # refresh page
            continue
        else:
            break
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    table_dates = []
    tables = soup.find_all('table', class_="a-bordered a-horizontal-stripes mojo-table releases-by-region")
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            data = row.find_all('td')
            if len(data) != 0:
                table_dates.append(data)
    new_list = []
    for list_ in table_dates:
        list_row = []
        for element in list_:
            if element.text == '\n        –\n    ':
                list_row.append('missing')
            elif element.text == '':
                list_row.append('missing')
            else: 
                list_row.append(element.text)
        new_list.append(list_row)

    original_release_df = pd.DataFrame(new_list, columns=['country','release_date','opening_revenue','gross_revenue'])
    original_release_df['release_date'].replace({"missing": "Jan 1, 2022"}, inplace=True)
    original_release_df['release_date'] = pd.to_datetime(original_release_df['release_date'], format='%b %d, %Y')
    original_release_df = original_release_df.sort_values(by="release_date")
    original_release_df.reset_index(drop=True)
    
    #Get earliest release location opening revenue
    try:
        raw_earliest_release_location_opening = original_release_df.opening_revenue[0]
        earliest_release_location_opening_gross = money_to_int(raw_earliest_release_location_opening)
    except:
        earliest_release_location_opening_gross = None 
    
    #Get earliest release location gross revenue
    try:
        raw_earliest_release_location_total_gross = original_release_df.gross_revenue[0]
        earliest_release_location_original_gross = money_to_int(raw_earliest_release_location_total_gross)
    except: 
        earliest_release_location_original_gross = None 
    
    #QUITTING CHROME DRIVER
    driver.quit()
    
    #Create movie dictionary and return
    movie_dict = dict(zip(headers, [title,
                                earliest_release_location_opening_gross,
                                earliest_release_location_original_gross]))

    return movie_dict

In [ ]:
fl_movies_page_info_list3 = [] #saving the info to a dictionary OUTSIDE the function

for link in earliestrelease_notdomestic_movies2.link_stub:
    fl_movies_page_info_list3.append(get_movie_dict(link))
    
fl_movies_page_info3 = pd.DataFrame(fl_movies_page_info_list3)  #converting list of dict to df
fl_movies_page_info3.set_index('title', inplace=True) 

import pickle #immediately pickling dataframe
fl_movies_page_info3.to_pickle('fl_movies_page_info3.pickle')